![ ](https://www.pon-cat.com/application/files/7915/8400/2602/home-banner.jpg)

# <center> **Визуализация данных** </center>

Визуализация занимает важную часть в анализе данных. Представляя информацию в графическом виде, вы облегчаете процесс ее восприятия, что дает возможность выделять дополнительные закономерности, оценивать соотношения величин, быстрее доносить ключевые аспекты в данных. 

Начнем с небольшой "памятки", о которой всегда нужно помнить при создании любых графиков. 

### <center> Как визуализировать данные и заставить всех тебя ненавидеть </center>

1. Заголовок графика для слабаков. По графику всегда понятно, какие данные и явления он описывает.
2. Ни в коем случае не подписывай ни одной оси у графика. Пусть смотрящий развивает свою интуицую! 
3. Единицы измерения совсем не обязательны. Какая разница, в чем измеряли количество - в людях или в литрах!
4. Чем меньше шрифт на графике, тем острее зрение смотрящего. 
5. На одном графике нужно стараться уместить всю информацию, которая у тебя есть в датасете. С полными названиями, расшифровками, сносками. Чем больше текста - тем информативнее!
6. При любой возможности используйте 3D и спецэффекты, пусть знают, что ты — прирожденный дизайнер. К тому же, так будет меньше визуальных искажений. 

В качестве примера, рассмотрим кейс с пандемией. Воспользуемся датасетом с оперативно обновляемой статистикой по коронавирусу (COVID-19), который выложен в открытом доступе на Kaggle: https://www.kaggle.com/imdevskp/corona-virus-report?select=covid_19_clean_complete.csv

Если серьезно, то обязательно посмотрите список рекомендованных материалов в конце ноутбука по правилам оформления графиков и работе с библиотеками для визуализации данных в Python. 

Основные библиотеки для визуализации в Python, которые нам сегодня понадобятся - это **matplotlib, seaborn, plotly**. 

In [ ]:
# Установим вспомогательные пакеты
!pip3 install plotly-express
!pip3 install nbformat==4.2.0
!pip install plotly

In [ ]:
import pandas as pd
import numpy as np
import pickle

# Модули для визуализации
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook
import seaborn as sns
import plotly
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

%config InlineBackend.figure_format = 'svg' # Графики в svg выглядят четче

# Увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 7, 5

import warnings
warnings.filterwarnings('ignore')

Считаем данные и посмотрим количество стран в датасете и какие дни он покрывает. 

In [ ]:
data = pd.read_csv('./data/covid_19_clean.csv')

print(f"Количество стран: {data['Country/Region'].nunique()}")
print(f"Дни с {min(data['Date'])} по {max(data['Date'])}, всего {data['Date'].nunique()} дней.")

# Выведем последнюю статистику по России 
display(data[data['Country/Region'] == 'Russia'].tail())

# Поменяем формат даты из str в pd.Timestamp 
data['Date'] = pd.to_datetime(data['Date'], format = '%Y-%m-%d')

Распространение COVID-19 - это наглядный пример экспоненциального распределения. Чтобы это продемонстрировать, построим график по общему, выздоровевших и умерших. Мы будем использовать линейный тип графика (**Line Chart**), которые может отражать динамику по одному или нескольким показателям. Его удобно использовать, чтобы посмотреть, как величина меняется во времени. 
Возьмем переменные *Confirmed* и *Deaths*. 

In [ ]:
# В pandas встроено построение графиков
data.Confirmed.plot(); # попробуйте поменять тип графика через аргумент kind

Линейный график по одной переменной: количество заболевших. 

In [ ]:
data[['Confirmed', 'Date']].groupby('Date').sum().plot();

Линейный график по двум переменным: количество заболевших и умерших. 

In [ ]:
data[['Confirmed', 'Deaths', 'Date']].groupby('Date').sum().plot();

In [ ]:
# Дополнительные настройки

ax = data[['Confirmed', 
           'Recovered', 
           'Date']].groupby('Date').sum().plot(title='Рост числа заболевших и выздоровевших от COVID-19')
ax.set_xlabel("Дата")
ax.set_ylabel("Количество заболевших"); # символ ';' прячет первую строку Text из графика

Код для аналогичного линейного графика на matplotlib, а не через pandas (обращаемся через plt к заимпортированному matplotlib.pyplot):

In [ ]:
plt.plot(data[['Confirmed', 'Deaths', 'Date']].groupby('Date').sum())
plt.title('Рост числа заболевших и выздоровевших от COVID-19')
plt.xlabel('Дата')
plt.ylabel('Количество заболевших')
plt.xticks(rotation=45)
plt.show()

График выше показывает нам общую информацию по всему миру. Давайте выделим 10 наиболее пострадавших стран (по итогам последнего дня из датасета) и на одном **Line Chart** покажем данные по каждой из них по числу зарегистрированных случаев болезни. В этот раз, попробуем воспользоваться библиотекой **plotly**. 

In [ ]:
# Выделим топ-10 стран по количеству подтвержденных случаев 
df_top = data[data['Date'] == max(data.Date)]
df_top = df_top.groupby('Country/Region', as_index=False)['Confirmed'].sum()
df_top = df_top.nlargest(10, 'Confirmed')

# Выделим тренд с учетом времени
df_trend = data.groupby(['Date','Country/Region'], as_index=False)['Confirmed'].sum()
df_trend = df_trend.merge(df_top, on='Country/Region')
df_trend.rename(columns={'Country/Region' : 'Countries', 
                         'Confirmed_x':'Cases',
                         'Date' : 'Dates'}, 
                inplace=True)

# px - это сокращения заимпортированного plotly_express
px.line(df_trend, 
        title='Рост числа выявленных случаев COVID-19',
        x='Dates', 
        y='Cases', 
        color='Countries')

Попробуем прологарифмировать признак количества выявленных случаев и построить график еще раз. 

In [ ]:
# Добавим столбец для визуализации логарифмического 
df_trend['ln(Cases)'] = np.log(df_trend['Cases'] + 1) # Добавляем 1 для случая log(0)

px.line(df_trend, 
        x='Dates', # обращение к столбцу, а не название осей
        y='ln(Cases)', 
        color='Countries', 
        title='COVID19 Рост числа заболевших в 10 наиболее пострадавших стран 10 (логарифмическая шкала)')

У большиснтва стран количество заболевших монотонно возрастает. 

Какие интересные выводы вы можете сделать по этому графику?

Попробуйте сделать аналогичные графики по количеству смертей от COVID-19 (по переменной **Deaths** вместо **Confirmed**) по топ-10 странам, наиболее пострадавшим по отношению количества умерших к количеству заболевших. 

In [ ]:
# TODO

Еще один тип графиков - круговая диаграмма (**Pie chart**). Чаще всего, этот график используют для визуализации отношения частей и целого (например, ваши траты за месяц в мобильном приложении банка), но многие его очень сильно недолюбливают (https://medium.com/@clmentviguier/the-hate-of-pie-charts-harms-good-data-visualization-cc7cfed243b6). 

Тем не менее, полезно знать, что они существуют, как они выглядят и как их построить. 

In [ ]:
# Простой и ужасный пример

df_trend.groupby('Countries')['Cases'].sum().plot(kind='pie');

In [ ]:
# Круговые диаграммы (пончик и пирог )

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
labels = [country for country in df_top['Country/Region']]

fig.add_trace(go.Pie(labels=labels, hole=0.3, hoverinfo="label+percent+name", 
                     values=[cases for cases in df_top.Confirmed], 
                     name="Доля", ), 1, 1)

fig.add_trace(go.Pie(labels=labels, pull=[0, 0, 0.2, 0], 
                     values=[cases for cases in df_top.Confirmed], 
                     name="Доля"), 1, 2)

fig.update_layout(
    title_text="Donut & Pie Chart: Распределение долей по заболевшим COVID-19 среди топ-10 стран",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text=' ', x=0.5, y=0.5, font_size=16, showarrow=False)],
    colorway=['rgb(69, 135, 24)', 'rgb(136, 204, 41)', 'rgb(204, 204, 41)', 
              'rgb(235, 210, 26)', 'rgb(209, 156, 42)', 'rgb(209, 86, 42)', 'rgb(209, 42, 42)', ])
fig.show()

На линейных графиках выше мы визуализировали совокупную информацию по странам по количеству выявленных заболевших. Теперь, давайте попробуем построить график по дневному тренду, посчитав разницу между текущим значением и значением предыдущего дня.
Для этой цели, воспользуемся гистограммой (**Histogram**). Также, добавим указатели по ключевым событиям, например, даты lockdown в провинции Ухань в Китае, Италии и Великобритании. 

In [ ]:
def add_daily_diffs(df):
    # 0 because the previous value is unknown
    df.loc[0,'Cases_daily'] = 0
    df.loc[0,'Deaths_daily'] = 0
    for i in range(1, len(df)):
        df.loc[i,'Cases_daily'] = df.loc[i,'Confirmed'] - df.loc[i - 1,'Confirmed']
        df.loc[i,'Deaths_daily'] = df.loc[i,'Deaths'] - df.loc[i - 1,'Deaths']
    return df

df_world = data.groupby('Date', as_index=False)['Deaths', 'Confirmed'].sum()
df_world = add_daily_diffs(df_world)

fig = go.Figure(data=[
    go.Bar(name='Количество заболевших',
           marker={'color': 'rgb(0,100,153)'},
           x=df_world.Date, 
           y=df_world.Cases_daily),
    go.Bar(name='Количество смертей', x=df_world.Date, y=df_world.Deaths_daily)
])

fig.update_layout(barmode='overlay', title='Статистика по числу заражений и летальных исходов от COVID-19 в мире',
                 annotations=[dict(x='2020-01-23', y=1797, text="Lockdown (Ухань)", 
                                   showarrow=True, arrowhead=1, ax=-100, ay=-200),
                              dict(x='2020-03-09', y=1797, text="Lockdown (Италия)", 
                                   showarrow=True, arrowhead=1, ax=-100, ay=-200),
                              dict(x='2020-03-23', y=19000, text="Lockdown (Великобритания)", 
                                   showarrow=True, arrowhead=1, ax=-100, ay=-200)])
fig.show()

In [ ]:
# Можно сохранить график в статичном виде на компьютер
plotly.offline.plot(fig, filename='my_beautiful_histogram.html', show_link=False)

Гистограмму часто путают со столбиковой диаграммой (Bar Chart) из-за визуального сходства, однако у этих графиков разные цели. Гиcтограмма пoказывает, как распределены данные в рамках непрерывного интервала или определенного периода времени. По вертикальной оси гистограммы находится частотность, по горизонтальной — интервалы или какой-то временной период.

Давайте теперь построим **Bar Chart**. Он бывает вертикальный и горизонтальный, выберем второй вариант. 
Построим график только для топ-20 стран по смертности. Будем рассчитывать эту статистику как отношение количества смертей к количеству подтвержденных заболевших по каждой стране. 

По некоторым странам в датасете статистика представлена по каждому региону (например, по всем штатам США). Для таких стран оставим только одно (максимальное) значение. Как вариант, можно было бы посчитать среднее по регионам и оставить его как показатель по стране.  

In [ ]:
# Столбчатая диаграмма

df_mortality = data.query('(Date == "2020-07-17") & (Confirmed > 100)') 
df_mortality['mortality'] = df_mortality['Deaths'] / df_mortality['Confirmed']
df_mortality['mortality'] = df_mortality['mortality'].apply(lambda x: round(x, 3))
df_mortality.sort_values('mortality', ascending=False, inplace=True)
# Оставим только максимальное значение смертности для стран, у которых статистика предоставлена по каждому региону

df_mortality.drop_duplicates(subset=['Country/Region'], keep='first', inplace=True)

fig = px.bar(df_mortality[:20].iloc[::-1],
             x='mortality', 
             y='Country/Region',
             labels={'mortality': 'Уровень смертности', 'Country_Region': 'Страна'},
             title=f'Уровень смертности: топ-20 стран по состоянию на 2020-04-17', 
             text='mortality', 
             height=800, 
             orientation='h') # горизонтальный
fig.show()

# TODO: раскрасить столбцы по тепловой карте (используя уровень смерности)
# Для этого добавьте аргументы color = 'mortality'

**Heat Maps** (тепловые карты) достаточно полезно использовать для дополнительная визуализации матриц корреляций между признаками. Когда признаков очень много, с помощью такого графика вы быстрее сможете оценить, какие признаки сильно скоррелировано или не обладают линейной взаимосвязью. 

In [ ]:
# Тепловая карта (используя seaborn) 
sns.heatmap(data.corr(), annot=True, fmt='.2f', cmap='cividis'); # попробуйте другой цвет, например, 'RdBu'

График рассеяния помогает найти взаимосвязь между двумя показателями. Для этого, можно воспользоваться pairplot, который сразу выведет гистограмму по каждой переменной и диаграмму рассеяния по двум переменным (по разным осям графика). 

In [ ]:
# Попарные двумерные распределений (pairplot)
sns_plot = sns.pairplot(data[['Deaths', 'Confirmed']])
sns_plot.savefig('pairplot.png') # сохранение картинки

При анализе данных, часто используют сводные таблицы. Сводная таблица (**Pivot table**) может автоматически сортировать, рассчитывать суммы или получить среднее значение из данных.

In [ ]:
# Сводные таблицы

plt.figure(figsize=(12, 4))
df_new = df_mortality.iloc[:10]
df_new['Confirmed'] = df_new['Confirmed'].astype(np.int)
df_new['binned_fatalities'] = pd.cut(df_new['Deaths'], 3)
platform_genre_sales = df_new.pivot_table(
                        index='binned_fatalities', 
                        columns='Country/Region', 
                        values='Confirmed', 
                        aggfunc=sum).fillna(int(0)).applymap(np.int)
sns.heatmap(platform_genre_sales, annot=True, fmt=".1f", linewidths=0.7, cmap="viridis");

Фоновая картограмма (choropleth map) как раз нужна для таких данных,которые необходимо показать по странам или регионам. Дополнительно, можно использовать временную ось, чтобы рассмотреть изменение показателей в динамике. 

In [ ]:
# Карта (фоновая картограмма)

# файл с аббревиатурами стран для матчинга с картой
with open('./data/countries_codes.pkl', 'rb') as file:
    countries_codes = pickle.load(file)
    
df_map = data.copy()
df_map['Date'] = data['Date'].astype(str)
df_map = df_map.groupby(['Date','Country/Region'], as_index=False)['Confirmed','Deaths'].sum()
df_map['iso_alpha'] = df_map['Country/Region'].map(countries_codes)
df_map['ln(Confirmed)'] = np.log(df_map.Confirmed + 1)
df_map['ln(Deaths)'] = np.log(df_map.Deaths + 1)

px.choropleth(df_map, 
              locations="iso_alpha", 
              color="ln(Confirmed)", 
              hover_name="Country/Region",
              hover_data=["Confirmed"],
              animation_frame="Date",
              color_continuous_scale=px.colors.sequential.OrRd,
              title = 'Total Confirmed Cases growth (Logarithmic Scale)')

Какую важную информацию дал новый график (визуализация по времени и геолокации)? Можно ли по графику ответить на вопросы:
* С какой страны началось распространение коронавируса?
* Какие страны наиболее пострадали от пандемии?
* На какую часть полушария приходится большинство случаев заболевания? Какие можно сформулировать гипотезы относительно температуры и скорости распространения вируса?

Какие еще наблюдения вы можете сделать из графика? 

### **Полезные материалы по правилам оформления графиков**
1. Twenty rules for good graphics https://robjhyndman.com/hyndsight/graphics/
2. Много статей и примеров https://www.darkhorseanalytics.com/blog .Например, как оформлять таблицы с резульатами https://www.darkhorseanalytics.com/portfolio/2016/1/7/data-looks-better-naked-clear-off-the-table


### **Полезные материалы по работе с библиотеками**

1. Руководство по библиотеке matplotlib https://matplotlib.org/3.2.1/tutorials/index.html 
2. Руководство по seaborn https://seaborn.pydata.org/tutorial.html
3. Документация plotly https://plotly.com/python/ 
4. [Kaggle COVID19-Explained through Visualizations](https://www.kaggle.com/anshuls235/covid19-explained-through-visualizations/#data)
5. Видео-лекция по визуализации из курса Open Data Science https://www.youtube.com/watch?v=uwQat1TV0JM

### **Дополнительные хорошие библиотеки python по визуализации**:
* Bokeh
* ggplot
* geoplotlib
* pygal
